In [1]:
import pandas as pd
import os
import codecs
import csv

import numpy as np
import matplotlib.pyplot as plt

import sys
import random
import copy

In [2]:
def readCsv(path):

    os.chdir(path)
    system = {}

    onlyfiles = [f for f in os.listdir() if os.path.isfile(f)]

    #print(onlyfiles)
    for f in onlyfiles:
        system[f] = pd.read_csv(f, encoding='utf8')

    return onlyfiles, system

In [7]:
def removeCols(df, col_lst : list):

    schp_year_cols = list(df.columns)
    for col in col_lst:
        selc_cols = schp_year_cols.remove(col)
    schp_year = df[schp_year_cols]

    return schp_year

In [3]:
nm, inf = readCsv(r"C:\Users\Usuario\Python_Notebooks\Mercurio\Country_View\shopfy_Country_Views")

In [5]:
origen = inf['visits_2020-01-01.csv']

In [6]:
origen.head()

,day,location_city,location_country,total_visitors,total_sessions,total_carts,total_checkouts,total_conversion,total_orders_placed
0,2020-01-01,Guissona,Spain,1,1,0,0,0.0,0
1,2020-01-01,Madrid,Spain,7,7,0,0,0.0,0
2,2020-01-01,Sint-truiden,Belgium,1,1,0,0,0.0,0
3,2020-01-01,Palma De Mallorca,Spain,1,1,0,0,0.0,0
4,2020-01-01,Vilnius,Lithuania,1,1,0,0,0.0,0


In [8]:
origen_wd = removeCols(origen, ['day'])

In [9]:
origen_wd.head()

,location_city,location_country,total_visitors,total_sessions,total_carts,total_checkouts,total_conversion,total_orders_placed
0,Guissona,Spain,1,1,0,0,0.0,0
1,Madrid,Spain,7,7,0,0,0.0,0
2,Sint-truiden,Belgium,1,1,0,0,0.0,0
3,Palma De Mallorca,Spain,1,1,0,0,0.0,0
4,Vilnius,Lithuania,1,1,0,0,0.0,0


In [10]:
#Agrupamos los registros por pais y ciudad para conocer los registros que podemos descartar.
origen_gr = origen_wd.groupby(["location_city","location_country"]).sum()

In [11]:
origen_gr.head()

,,total_visitors,total_sessions,total_carts,total_checkouts,total_conversion,total_orders_placed
location_city,location_country,,,,,,
's Gravenmoer,Netherlands,1,1,0,0,0.0,0
's-graveland,Netherlands,2,2,0,0,0.0,0
's-gravendeel,Netherlands,1,1,0,0,0.0,0
's-gravenhage,Netherlands,23,24,0,0,0.0,0
's-gravenpolder,Netherlands,1,1,0,0,0.0,0


In [12]:
msk_city_import = origen_gr["total_orders_placed"] != 0
city_import = origen_gr[msk_city_import]

In [13]:
city_import.head()

,,total_visitors,total_sessions,total_carts,total_checkouts,total_conversion,total_orders_placed
location_city,location_country,,,,,,
's-hertogenbosch,Netherlands,266,277,13,9,1.76666,5
A Coruna,Spain,37,39,2,2,1.50000,2
A Coruña,Spain,101,103,1,1,1.00000,1
Aalsmeer,Netherlands,48,50,1,2,0.83333,2
Aarschot,Belgium,23,29,1,1,0.14285,1


In [23]:
list_ciudades = list(city_import.index.get_level_values("location_city"))

In [31]:
#Vamos a generar un CSV unicamente con las ciudades que han realiazado alguna compra en 2020
msk_out_csv = origen.location_city.isin(list_ciudades)
out_csv = origen[msk_out_csv]

In [32]:
print("Data origen tenia %d regstros" % len(origen))
print("Data filtrada tenia %d regstros" % len(out_csv))

Data origen tenia 92346 regstros
Data filtrada tenia 27196 regstros


In [36]:
csv.register_dialect('myDialect',
                     delimiter=';',
                     quoting=csv.QUOTE_ALL)
with open(r'anual_filter_city.csv', 'w', newline='') as file:
    writer = csv.writer(file, dialect='myDialect')
    writer.writerows(out_csv)

In [55]:
def writeCsv(data):
        os.chdir(r"C:\Users\Usuario\Python_Notebooks\Mercurio\Country_View\shopfy_Country_Views_Out")
        data.to_csv('anual_filter_city.csv', sep=";",index = False)

In [56]:
writeCsv(out_csv)